<a href="https://colab.research.google.com/github/ima-projects/Movie_Recommendation_Sys_using_Machine_Learning/blob/main/Movie_Recommendation_Sys_using_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the dependencies

In [18]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


Data Collection and Pre Processing

In [28]:
# loading the data from the csv file to a pandas dataframe
movies_data = pd.read_csv('/content/movies.csv', error_bad_lines=False, engine="python")

<ipython-input-28-4565235ff1f9>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  movies_data = pd.read_csv('/content/movies.csv', error_bad_lines=False, engine="python")
Skipping line 1932: unexpected end of data


In [26]:
# printing the first 5 rows of the dataframe
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [38]:
# number of rows and columns in the dataframe

movies_data.shape

(1930, 24)

In [39]:
# selecting the relevant features for recommendation

selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [40]:
# replacing the null values (missing values) with null string

for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [41]:
# combining all the 5 selected features

combined_features = movies_data['genres']+''+movies_data['keywords']+''+movies_data['tagline']+''+movies_data['cast']+''+movies_data['director']

In [42]:
print(combined_features)

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
1925    Drama Thrillerholiday prison thailand drug inj...
1926    Horror ThrillerFear The Demon That Doesn't Fea...
1927    Comedy Romance Dramarole reversal mistaken ide...
1928    Drama Thrillersouthern usa rape bar wife husba...
1929    Comedy Dramaflying 1970s false identity biogra...
Length: 1930, dtype: object


In [43]:
# converting the text data to feature vectors (producing a numerical value)

vectorizer = TfidfVectorizer()

In [44]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [45]:
print(feature_vectors)

  (0, 1739)	0.1732832370604628
  (0, 9954)	0.2641441125259753
  (0, 7935)	0.15967901558833805
  (0, 6938)	0.2408875423093321
  (0, 11176)	0.17218306838307149
  (0, 12818)	0.2101440287492308
  (0, 10758)	0.21367909936508256
  (0, 10296)	0.2176309720926889
  (0, 13309)	0.20694616839410457
  (0, 13139)	0.22211123587474693
  (0, 10314)	0.15682178900687538
  (0, 8865)	0.25053989105385055
  (0, 8666)	0.09182709195712918
  (0, 13127)	0.12257060551723051
  (0, 11635)	0.07667997159567357
  (0, 10924)	0.2641441125259753
  (0, 2253)	0.2408875423093321
  (0, 12696)	0.13255351987674138
  (0, 10989)	0.32560479671580106
  (0, 4887)	0.17218306838307149
  (0, 2152)	0.2408875423093321
  (0, 4459)	0.2641441125259753
  (0, 10448)	0.09525269383470111
  (0, 4239)	0.10913125975044437
  (0, 211)	0.07970236276519012
  :	:
  (1929, 5411)	0.2118174049279699
  (1929, 12320)	0.2037760590390897
  (1929, 5042)	0.19457105212230053
  (1929, 5461)	0.22972351619443898
  (1929, 5005)	0.20040482263523168
  (1929, 7573)	0.

Cosine Similarity

In [46]:
# getting the similarity scores using cosine similarity (producing a numerical similarity score value)

similarity = cosine_similarity(feature_vectors)

In [47]:
print(similarity)

[[1.         0.06521872 0.01174355 ... 0.00771006 0.         0.01925605]
 [0.06521872 1.         0.02542042 ... 0.01356424 0.07247026 0.04374193]
 [0.01174355 0.02542042 1.         ... 0.         0.         0.        ]
 ...
 [0.00771006 0.01356424 0.         ... 1.         0.         0.05492995]
 [0.         0.07247026 0.         ... 0.         1.         0.        ]
 [0.01925605 0.04374193 0.         ... 0.05492995 0.         1.        ]]


In [48]:
print(similarity.shape)

(1930, 1930)


Getting the movie name from user

In [49]:
# getting the movie name from the user (input)

movie_name = input(' Enter your favorite movie name: ')

 Enter your favorite movie name: iron man


In [55]:
# creating a list with all the movie names given in the dataset (to compare with the value given by user input)

list_of_all_titles = movies_data['title'].tolist()
first_ten_titles = list_of_all_titles[:10]

for title in first_ten_titles:
    print(title)

Avatar
Pirates of the Caribbean: At World's End
Spectre
The Dark Knight Rises
John Carter
Spider-Man 3
Tangled
Avengers: Age of Ultron
Harry Potter and the Half-Blood Prince
Batman v Superman: Dawn of Justice


In [51]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [52]:
# finding the closest/most similar match in the dataset

close_match = find_close_match[0]
print(close_match)

Iron Man


In [53]:
# finding the index of the (most similar) movie with title

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

68


In [54]:
# get a list of similar movies (compared to the one most similar movie i.e. Iron Man which was found in the dataset)

similarity_score = list(enumerate(similarity[index_of_the_movie]))
first_ten_scores = similarity_score[:10]

for score in first_ten_scores:
    print(first_ten_scores)

[(0, 0.012946954286874231), (1, 0.035109046432793264), (2, 0.004721167880254073), (3, 0.005016222666173034), (4, 0.012415938959232146), (5, 0.004610648680203248), (6, 0.05745755019865102), (7, 0.13893570045682588), (8, 0.0), (9, 0.03043443641849399)]
[(0, 0.012946954286874231), (1, 0.035109046432793264), (2, 0.004721167880254073), (3, 0.005016222666173034), (4, 0.012415938959232146), (5, 0.004610648680203248), (6, 0.05745755019865102), (7, 0.13893570045682588), (8, 0.0), (9, 0.03043443641849399)]
[(0, 0.012946954286874231), (1, 0.035109046432793264), (2, 0.004721167880254073), (3, 0.005016222666173034), (4, 0.012415938959232146), (5, 0.004610648680203248), (6, 0.05745755019865102), (7, 0.13893570045682588), (8, 0.0), (9, 0.03043443641849399)]
[(0, 0.012946954286874231), (1, 0.035109046432793264), (2, 0.004721167880254073), (3, 0.005016222666173034), (4, 0.012415938959232146), (5, 0.004610648680203248), (6, 0.05745755019865102), (7, 0.13893570045682588), (8, 0.0), (9, 0.0304344364184939

In [56]:
len(similarity_score)

1930

In [57]:
# sorting the movies based on their similarity score by descending order (highest and lowest)
# x represents the first value of each element (the index i.e. 0,1,2...) in similarity_score
# x[1] represents the second value in each element (the similarity score i.e. 0.015,0.034,...) in similarity_score
# it is being sorted by x[1] here.

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(68, 1.0), (79, 0.22642561519100804), (31, 0.21250194902138048), (16, 0.17714773699679107), (26, 0.1521536430952489), (7, 0.13893570045682588), (353, 0.11537790666037616), (33, 0.11256136116686208), (46, 0.10906714477335187), (1210, 0.10558502451138016), (511, 0.10155979765267609), (954, 0.09979333208977206), (203, 0.09884038748551768), (1406, 0.09676163836717985), (174, 0.09406678980694108), (882, 0.09335819925927033), (85, 0.09190392645777257), (307, 0.09183221361188353), (64, 0.0898665860811958), (38, 0.08654339549447063), (101, 0.08640098182594488), (1282, 0.08425884156505582), (783, 0.08419194516584094), (356, 0.08097454345876658), (788, 0.08064295717633159), (205, 0.08016214438310043), (1553, 0.07940141685556665), (39, 0.07757021479328036), (1015, 0.07435716116119694), (182, 0.07405081987593261), (1654, 0.07169715413591257), (1651, 0.07151702217460196), (1864, 0.07127570933255788), (1664, 0.07119879393909671), (436, 0.0701917791860072), (421, 0.0701632766596687), (1029, 0.070076

In [58]:
# print the name of similar movies based on the index
# previously the title of the movie was used in order to find the index and now I will use the index of the movie to to find the title name (so opposite way)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0] # 0 represents the first value of the element from sorted_similar_movies (the index e.g. 68)
  title_from_index = movies_data[movies_data.index == index]['title'].values[0]
  
  # first 30 movies most similar to 'Iron Man' movie inside for loop
  if (i<31):
    print(i, '.',title_from_index) # # '.' is used to separate printed list integer from index and title below
    i+=1


Movies suggested for you : 

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . The Avengers
5 . Captain America: Civil War
6 . Avengers: Age of Ultron
7 . Tropic Thunder
8 . X-Men: The Last Stand
9 . X-Men: Days of Future Past
10 . Gothika
11 . X-Men
12 . The Judge
13 . X2
14 . The Nativity Story
15 . The Incredible Hulk
16 . Lucky You
17 . Captain America: The Winter Soldier
18 . The Expendables 3
19 . X-Men: Apocalypse
20 . The Amazing Spider-Man 2
21 . X-Men: First Class
22 . The Hunting Party
23 . Mortdecai
24 . Sherlock Holmes
25 . Deadpool
26 . Sherlock Holmes: A Game of Shadows
27 . Se7en
28 . TRON: Legacy
29 . K-PAX
30 . Ant-Man


Movie Recommendation System

In [59]:
# incorporating all the code together

movie_name = input(' Enter your favorite movie name: ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0] # 0 represents the first value of the element from sorted_similar_movies (the index e.g. 68)
  title_from_index = movies_data[movies_data.index == index]['title'].values[0]
  
  # first 30 movies most similar to 'Iron Man' movie inside for loop
  if (i<31):
    print(i, '.',title_from_index) # '.' is used to separate printed list integer from index and title below
    i+=1


 Enter your favorite movie name: avatar
Movies suggested for you : 

1 . Avatar
2 . Guardians of the Galaxy
3 . Alien³
4 . Clash of the Titans
5 . Moonraker
6 . The Right Stuff
7 . Galaxy Quest
8 . Star Trek Beyond
9 . Shadow Conspiracy
10 . Star Trek Into Darkness
11 . Pocahontas
12 . The Book of Life
13 . The Astronaut's Wife
14 . Snow White: A Tale of Terror
15 . Wing Commander
16 . Terminator Salvation
17 . Sunshine
18 . Planet of the Apes
19 . Vantage Point
20 . Gettysburg
21 . Alien: Resurrection
22 . Star Trek
23 . The Iron Giant
24 . Supernova
25 . Treasure Planet
26 . Colombiana
27 . Men in Black II
28 . Men in Black
29 . Memoirs of an Invisible Man
30 . The Losers
